<a href="https://colab.research.google.com/github/DaeSeokSong/image-processing/blob/feature%2FPreprocessing/Preprocessor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install**

# **Import**

In [1]:
# Image processing
import cv2
import numpy as np

from google.colab.patches import cv2_imshow
from google.colab import output

# ETC
import os
import time
import math

from sklearn.model_selection import train_test_split

## Mount google drive

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%cd /content/gdrive/MyDrive/Models/GAN_Scar
!ls -al

[Errno 2] No such file or directory: '/content/gdrive/MyDrive/Models/GAN_Scar'
/content
total 20
drwxr-xr-x 1 root root 4096 Sep 15 11:21 .
drwxr-xr-x 1 root root 4096 Sep 15 11:17 ..
drwxr-xr-x 4 root root 4096 Sep  9 13:21 .config
drwx------ 5 root root 4096 Sep 15 11:21 gdrive
drwxr-xr-x 1 root root 4096 Sep  9 13:22 sample_data


# **Grobal variable**

In [4]:
# Path
MODEL_PATH = "/content/gdrive/MyDrive/Models/Surgical-Wound_Segmentation"

RAW_DATA_PATH = "/Raw Dataset"

WOUND_TRAIN_PATH = "/Wound/train"
WOUND_TEST_PATH = "/Wound/test"
CVC_INPUT_PATH = "/CVC-clinicDB/Original"
CVC_GT_PATH = "/CVC-clinicDB/Ground Truth"

DATASET_PATH = "/Dataset"

IMAGES_PATH = '/images'
LABELS_PATH = '/labels'

# Image preprocess
NORM_INPUT_W_SIZE = 224
NORM_INPUT_H_SIZE = 224

# **Function**

In [6]:
def imshow_waitkey_enter(image):
    cv2_imshow(image)

    time.sleep(0.5)
    
    input("Please press the Enter key to proceed\n")
    output.clear()

    pass

# **Make dir**

In [7]:
# Create processed dataset dir
train_dir = os.path.join(MODEL_PATH + DATASET_PATH, 'train')
val_dir = os.path.join(MODEL_PATH + DATASET_PATH, 'val')
test_dir = os.path.join(MODEL_PATH + DATASET_PATH, 'test')

if not os.path.exists(train_dir):
    os.makedirs(train_dir)

if not os.path.exists(val_dir):
    os.makedirs(val_dir)

if not os.path.exists(test_dir):
    os.makedirs(test_dir)

<br>
<br>
<br>
<br>
<br>
<br>

# ***Preprocess Performace Compare Test***


---

<br>

***공통사항***
* 원본 이미지 크기 = 224 * 224
* epoch = 100
* batch = 8
> UNet batch size = 4<br>
> Wound dataset batch size = 8
* lr = 1e-3
* Train:Val = 7:3
* Used model = Polar Res UNet++ (ISIC 2018 dice scroe rank no.1)

<br>

***주의사항***
* 이미지의 사이즈는 정규화 되어야 한다.
* Convolution에서 특징점을 잘 추출하기 때문에 색상영역이나, 스무딩/샤프닝 같은 필터 처리는 되려 성능저하 요소가 될 수 있다. 

<br>

***참고사항***
* CNN 대표 모델들의 네트워크 입력 사이즈는 224 * 224
* 이미지의 사이즈가 큰 경우에, Overlap-Tite(down sampling) 전략으로 한 이미지를 나눠서 학습시킨다.

<br>

***현재 결과***

---




<br>
<br>
<br>
<br>
<br>
<br>

## [***Control Group***](https://www.nature.com/articles/s41598-020-78799-w)
> [Train code](https://github.com/uwm-bigdata/wound-segmentation/blob/de3b9c00974b065b2062fd5cd2efe670da1f8f51/train.py)
> <br>
> [Data generator](https://github.com/uwm-bigdata/wound-segmentation/blob/de3b9c00974b065b2062fd5cd2efe670da1f8f51/utils/io/data.py#L10)
> <br>
> <br>
> Raw dataset에 대한 전처리 없이, load 후 model에 들어가기 전에 Normalize
> <br>
> * 현재까지 대조군 성능을 넘은 실험군이 없다.

```
model = Deeplabv3(input_shape=(input_dim_x, input_dim_y, 3), classes=1)
```

### Performance

<br>

#### **Grayscale**

```
Input size =  torch.Size([8, 1, 224, 224])

TRAIN || EPOCH 0100 | BATCH  069 /  073 | DICE LOSS  0.1168
TRAIN || EPOCH 0100 | BATCH  070 /  073 | DICE LOSS  0.1192
TRAIN || EPOCH 0100 | BATCH  071 /  073 | DICE LOSS  0.1188
TRAIN || EPOCH 0100 | BATCH  072 /  073 | DICE LOSS  0.1189
TRAIN || EPOCH 0100 | BATCH  073 /  073 | DICE LOSS  0.1205

VALID || EPOCH 0100 | BATCH  028 /  032 | DICE LOSS  0.1912
VALID || EPOCH 0100 | BATCH  029 /  032 | DICE LOSS  0.1905
VALID || EPOCH 0100 | BATCH  030 /  032 | DICE LOSS  0.1893
VALID || EPOCH 0100 | BATCH  031 /  032 | DICE LOSS  0.1957
VALID || EPOCH 0100 | BATCH  032 /  032 | DICE LOSS  0.1981

Best loss in Train =  0.1101
Best loss in Validation =  0.1374
```

##### ***Train/Val Graph***

<br>

<img src = "https://drive.google.com/uc?id=1AnFMpx_n_TaeY5VnL4f5ztw3uo-HmLLq"  width = 320>

<br>

---

<br>

##### ***Train/Val result image***

<br>

<img src = "https://drive.google.com/uc?id=1Ek3gu_fT0nKRsxU48MY-v2kvGIDXtSIp"  width = 640>

#### **Color**

```
Input size =  torch.Size([8, 3, 224, 224])

TRAIN || EPOCH 0100 | BATCH  069 /  073 | DICE LOSS  0.1172
TRAIN || EPOCH 0100 | BATCH  070 /  073 | DICE LOSS  0.1171
TRAIN || EPOCH 0100 | BATCH  071 /  073 | DICE LOSS  0.1168
TRAIN || EPOCH 0100 | BATCH  072 /  073 | DICE LOSS  0.1162
TRAIN || EPOCH 0100 | BATCH  073 /  073 | DICE LOSS  0.1158

VALID || EPOCH 0100 | BATCH  028 /  032 | DICE LOSS  0.1384
VALID || EPOCH 0100 | BATCH  029 /  032 | DICE LOSS  0.1382
VALID || EPOCH 0100 | BATCH  030 /  032 | DICE LOSS  0.1380
VALID || EPOCH 0100 | BATCH  031 /  032 | DICE LOSS  0.1391
VALID || EPOCH 0100 | BATCH  032 /  032 | DICE LOSS  0.1374

Best loss in Train =  0.1017
Best loss in Validation =  0.1102
```

##### ***Train/Val Graph***

<br>

<img src = "https://drive.google.com/uc?id=1Pwo0j2qVIk6RBU3JS_aCIa27lRRN4od2"  width = 320>

<br>

---

<br>

##### ***Train/Val result image***

<br>

<img src = "https://drive.google.com/uc?id=1XO3EJEbRbi_HEB7QpsuX-L-RhpE7K1Fk"  width = 640>

### Arrange dataset
> Not change anything about input image(scar)

In [ ]:
# Set load image dir path
train_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + IMAGES_PATH
train_label_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + LABELS_PATH
test_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + IMAGES_PATH
test_label_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + LABELS_PATH

# Load raw input images for Train
inputs_train = os.listdir(train_image_path)
inputs_train.sort()

# Load raw label images for Train
labels_train = os.listdir(train_label_path)
labels_train.sort()

# Load raw input images for Test
input_test = os.listdir(test_image_path)
input_test.sort()

# Load raw label images for Test
label_test = os.listdir(test_label_path)
label_test.sort()

# Split train:val:test = 7:3
input_train, input_val, label_train, label_val = train_test_split(inputs_train, labels_train, test_size=0.3, random_state=1)

# Merge train/val/test set
train_dataset = dict(zip(input_train, label_train))
val_dataset = dict(zip(input_val, label_val))
test_dataset = dict(zip(input_test, label_test))

# ================================ Train ================================
# Init image index
idx = 0

# Create preprocessed scar image
for input, label in train_dataset.items(): 
    # Access input image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(train_label_path)
    label_image = cv2.imread(label)

    # Save normalized image
    cv2.imwrite(os.path.join(train_dir, f'scar_{idx:03d}.png'), image)
    cv2.imwrite(os.path.join(train_dir, f'label_{idx:03d}.png'), label_image)
    idx += 1

# ============================== Validation ==============================
# Init image index
idx = 0

# Create preprocessed scar image
for input, label in val_dataset.items(): 
    # Access input image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(train_label_path)
    label_image = cv2.imread(label)

    # Save normalized image
    cv2.imwrite(os.path.join(val_dir, f'scar_{idx:03d}.png'), image)
    cv2.imwrite(os.path.join(val_dir, f'label_{idx:03d}.png'), label_image)
    idx += 1

# ================================= Test =================================
# Init image index
idx = 0

# Create preprocessed scar image
for input, label in test_dataset.items(): 
    # Access input image
    os.chdir(test_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(test_label_path)
    label_image = cv2.imread(label)

    # Save normalized image
    cv2.imwrite(os.path.join(test_dir, f'scar_{idx:03d}.png'), image)
    cv2.imwrite(os.path.join(test_dir, f'label_{idx:03d}.png'), label_image)
    idx += 1

<br>
<br>
<br>
<br>
<br>
<br>

## ***Experimental Group***

### Case 1

1. Delete Black Boundary(Padding)
2. Resize Scar Image to (224, 224)

In [ ]:
# Set load image dir path
train_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + IMAGES_PATH
train_label_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + LABELS_PATH
test_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + IMAGES_PATH
test_label_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + LABELS_PATH

# Load raw input images for Train
inputs_train = os.listdir(train_image_path)
inputs_train.sort()

# Load raw label images for Train
labels_train = os.listdir(train_label_path)
labels_train.sort()

# Load raw input images for Test
input_test = os.listdir(test_image_path)
input_test.sort()

# Load raw label images for Test
label_test = os.listdir(test_label_path)
label_test.sort()

# Split train:val:test = 7:3
input_train, input_val, label_train, label_val = train_test_split(inputs_train, labels_train, test_size=0.3, random_state=1)

# Merge train/val/test set
train_dataset = dict(zip(input_train, label_train))
val_dataset = dict(zip(input_val, label_val))
test_dataset = dict(zip(input_test, label_test))

# ================================ Train ================================
# Init image index
idx = 0

# Create preprocessed scar image
for input, label in train_dataset.items(): 
    # Access input image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(train_label_path)
    label_image = cv2.imread(label)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # 2. Resize image to (224, 224)
    # Resize input image
    image = cv2.resize(image,
                       (NORM_INPUT_W_SIZE, NORM_INPUT_H_SIZE),
                       interpolation=cv2.INTER_CUBIC)
    
    # Resize label image
    label_image = cv2.resize(label_image, 
                             (NORM_INPUT_W_SIZE, NORM_INPUT_H_SIZE),
                             interpolation=cv2.INTER_CUBIC)

    # Save normalized image
    cv2.imwrite(os.path.join(train_dir, f'scar_{idx:03d}.png'), image)
    cv2.imwrite(os.path.join(train_dir, f'label_{idx:03d}.png'), label_image)
    idx += 1

# ============================== Validation ==============================
# Init image index
idx = 0

# Create preprocessed scar image
for input, label in val_dataset.items(): 
    # Access input image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(train_label_path)
    label_image = cv2.imread(label)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # 2. Resize image to (224, 224)
    # Resize input image
    image = cv2.resize(image,
                       (NORM_INPUT_W_SIZE, NORM_INPUT_H_SIZE),
                       interpolation=cv2.INTER_CUBIC)
    
    # Resize label image
    label_image = cv2.resize(label_image, 
                             (NORM_INPUT_W_SIZE, NORM_INPUT_H_SIZE),
                             interpolation=cv2.INTER_CUBIC)

    # Save normalized image
    cv2.imwrite(os.path.join(val_dir, f'scar_{idx:03d}.png'), image)
    cv2.imwrite(os.path.join(val_dir, f'label_{idx:03d}.png'), label_image)
    idx += 1

# ================================= Test =================================
# Init image index
idx = 0

# Create preprocessed scar image
for input, label in test_dataset.items(): 
    # Access input image
    os.chdir(test_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(test_label_path)
    label_image = cv2.imread(label)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # 2. Resize image to (224, 224)
    # Resize input image
    image = cv2.resize(image,
                       (NORM_INPUT_W_SIZE, NORM_INPUT_H_SIZE),
                       interpolation=cv2.INTER_CUBIC)
    
    # Resize label image
    label_image = cv2.resize(label_image, 
                             (NORM_INPUT_W_SIZE, NORM_INPUT_H_SIZE),
                             interpolation=cv2.INTER_CUBIC)

    # Save normalized image
    cv2.imwrite(os.path.join(test_dir, f'scar_{idx:03d}.png'), image)
    cv2.imwrite(os.path.join(test_dir, f'label_{idx:03d}.png'), label_image)
    idx += 1

### Case 2

1. Delete Black Boundary(Padding)
2. Pad the rest part to (224, 224) based on the middle of the image

In [ ]:
# Set load image dir path
train_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + IMAGES_PATH
train_label_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + LABELS_PATH
test_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + IMAGES_PATH
test_label_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + LABELS_PATH

# Load raw input images for Train
inputs_train = os.listdir(train_image_path)
inputs_train.sort()

# Load raw label images for Train
labels_train = os.listdir(train_label_path)
labels_train.sort()

# Load raw input images for Test
input_test = os.listdir(test_image_path)
input_test.sort()

# Load raw label images for Test
label_test = os.listdir(test_label_path)
label_test.sort()

# Split train:val:test = 7:3
input_train, input_val, label_train, label_val = train_test_split(inputs_train, labels_train, test_size=0.3, random_state=1)

# Merge train/val/test set
train_dataset = dict(zip(input_train, label_train))
val_dataset = dict(zip(input_val, label_val))
test_dataset = dict(zip(input_test, label_test))

# ================================ Train ================================
# Init image index
idx = 0

# Create preprocessed scar image
for input, label in train_dataset.items(): 
    # Access input image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(train_label_path)
    label_image = cv2.imread(label)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # 2. Pad image to (224, 224)
    # Calculate padding size
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    # Modify error
    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
    
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    # Resizing
    image = cv2.copyMakeBorder(image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])


    # Save normalized image
    cv2.imwrite(os.path.join(train_dir, f'scar_{idx:03d}.png'), image)
    cv2.imwrite(os.path.join(train_dir, f'label_{idx:03d}.png'), label_image)
    idx += 1

# ============================== Validation ==============================
# Init image index
idx = 0

# Create preprocessed scar image
for input, label in val_dataset.items(): 
    # Access input image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(train_label_path)
    label_image = cv2.imread(label)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # 2. Pad image to (224, 224)
    # Calculate padding size
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    # Modify error
    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
    
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    # Resizing
    image = cv2.copyMakeBorder(image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    cv2.imwrite(os.path.join(val_dir, f'scar_{idx:03d}.png'), image)
    cv2.imwrite(os.path.join(val_dir, f'label_{idx:03d}.png'), label_image)
    idx += 1

# ================================= Test =================================
# Init image index
idx = 0

# Create preprocessed scar image
for input, label in test_dataset.items(): 
    # Access input image
    os.chdir(test_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(test_label_path)
    label_image = cv2.imread(label)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # 2. Pad image to (224, 224)
    # Calculate padding size
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    # Modify error
    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
    
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    # Resizing
    image = cv2.copyMakeBorder(image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    cv2.imwrite(os.path.join(test_dir, f'scar_{idx:03d}.png'), image)
    cv2.imwrite(os.path.join(test_dir, f'label_{idx:03d}.png'), label_image)
    idx += 1

### Case 3

1. Delete Black Boundary(Padding)
2. Resize Scar Image to (224, 224) in proportion to the maximum ratio
3. Pad the rest part to (224, 224) in the direction of right/down 

### Case 4

1. Delete Black Boundary(Padding)
2. Resize Scar Image to (224, 224) in proportion to the maximum ratio
3. Pad the rest part to (224, 224) based on the middle of the image

### Case 5

1. Delete Black Boundary(Padding)
2. Resize Scar Image to 224 based on the smaller size(row or column), keeping the proportion
3. Sampling by moving the kernel both on input and label, Then merge it to fit the label and input pair